# Usage of DataSynthesizer

> DataSynthesizer is developed in Python 3.5.2 and with some third-party modules, including
- NumPy, SciPy, Pandas, dateutil (available in Anaconda 4.3.0)
- faker https://github.com/joke2k/faker

Given a private dataset, DataSynthesizer can generate a synthetic dataset for release to public. It infers data types and domains of the attributes in dataset. Histograms are used to model the distribution of each attribute. Synthetic dataset is sampled from the histograms or uniformly sampled from the inferred domains. It also applies differential privacy to the histograms before sampling from them.

## Data types
 The DataSynthesizer currently supports 4 basic data types.

| data type | example |
|-----------|---------|
| integer   | id, age, ...|
| float     | score, rating, ...|
| string    | first name, gender, ...|
| datetime  | birthday, event time, ...|

## Data description format

The domain of an attribute is as follows.
- The "catagorical" indicates attributes with particular values, e.g., "gender", "nationality".
- Domains are modeled by histograms, except noncategorical "string".

|data type|categorical  |min             |max             |values             |probabilities      |values count      |missing rate|
|---------|----------|----------------|----------------|-------------------|-------------------|------------------|------------|
|int      |True/False|min             |max             |x-axis in histogram|y-axis in histogram|#bins in histogram|missing rate|
|float    |True/False|min             |max             |x-axis in histogram|y-axis in histogram|#bins in histogram|missing rate|
|string   |   True   |min in length   |max in length   |x-axis in histogram|y-axis in histogram|#bins in histogram|missing rate|
|string   |   False  |min in length   |max in length   |0                  |0                  |0               |missing rate|
|datetime |True/False|min in timestamp|max in timestamp|x-axis in histogram|y-axis in histogram|#bins in histogram|missing rate|

##### Step 0: Import DataDestriber and DataGenerator from DataSynthesizer

In [1]:
from DataSynthesizer import DataDestriber, DataGenerator

In [2]:
# input and output files
input_dataset_file = './raw_data/AdultIncomeData/adult.csv'
dataset_description_file = './output/description/AdultIncomeData_description.csv'
synthetic_dataset_file = './output/synthetic_data/AdultIncomeData_synthetic.csv'

##### Step 1: Initialize a DatasetDescriber

In [3]:
describer = DataDestriber()

##### Step 2: Generate dataset description

The dataset description is inferred by code, while users can also customize the data types and categorical indicators, e.g.,
    - "education-num" is of type "float".
    - "native-country" is not categrocial.
    - "age" is categorical.

In [4]:
describer.describe_dataset(file_name=input_dataset_file,
                           column_to_datatype_dict={'education-num': 'float'},
                           column_to_categorical_dict={'native-country':False,'age':True})

##### Step 3: Get the dataset description

Let's take a look at the input dataset

In [5]:
describer.input_dataset.head()

,ID,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,1.0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,NaN,White,Male,2174.0,NaN,40.0,United-States,<=50K
1,2.0,50.0,Self-emp-not-inc,83311.0,Bachelors,NaN,NaN,NaN,Husband,White,NaN,0.0,0.0,13.0,United-States,NaN
2,3.0,38.0,Private,215646.0,HS-grad,9.0,Divorced,NaN,NaN,White,Male,NaN,NaN,NaN,United-States,<=50K
3,4.0,53.0,NaN,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,5.0,28.0,NaN,NaN,Bachelors,NaN,NaN,NaN,NaN,Black,Female,0.0,NaN,40.0,Cuba,NaN


The dataset description is

In [6]:
describer.dataset_description

,data type,categorical,min,max,values,value counts,histogram size,missing
column,,,,,,,,
ID,int,False,1.0,32561.0,"[14653.0, 29305.0, 30933.0, 4885.0, 8141.0, 32...","[1120, 1114, 1111, 1101, 1098, 1098, 1093, 109...",20.0,0.335432
age,int,True,17.0,90.0,"[36.0, 31.0, 35.0, 33.0, 34.0, 23.0, 37.0, 28....","[812, 803, 803, 794, 791, 790, 773, 773, 772, ...",73.0,0.098676
workclass,string,True,2.0,17.0,"[ Private, Self-emp-not-inc, Local-gov, ?, ...","[13426, 1468, 1250, 1090, 762, 667, 586, 8, 5]",9.0,0.408433
fnlwgt,int,False,12285.0,1455435.0,"[156600.0, 84442.5, 228757.5, 10841.85, 300915...","[8379, 6399, 3447, 3375, 1996, 872, 287, 113, ...",20.0,0.232395
education,string,True,4.0,13.0,"[ HS-grad, Some-college, Bachelors, Masters...","[7195, 4914, 3638, 1185, 939, 790, 724, 619, 4...",16.0,0.319646
education-num,float,True,1.0,16.0,"[9.0, 10.0, 13.0, 14.0, 11.0, 7.0, 12.0, 6.0, ...","[9203, 6380, 4682, 1521, 1212, 1029, 926, 817,...",16.0,0.124321
marital-status,string,True,8.0,22.0,"[ Married-civ-spouse, Never-married, Divorce...","[11715, 8381, 3450, 808, 792, 323, 22]",7.0,0.217131
occupation,string,True,2.0,18.0,"[ Craft-repair, Exec-managerial, Prof-specia...","[2374, 2368, 2343, 2184, 2136, 1845, 1141, 104...",15.0,0.424741
relationship,string,True,5.0,15.0,"[ Husband, Not-in-family, Own-child, Unmarr...","[10173, 6343, 3874, 2667, 1202, 771]",6.0,0.231289


##### Step 4: save the dataset description

In [7]:
describer.dataset_description.to_csv(dataset_description_file)

### Generate synthetic data

##### Step 5: Initialize a DataGenerator.

In [8]:
generator = DataGenerator()

##### Step 6: Generate sysnthetic dataset

By default, the data is sampled from the histograms in dataset description. But users can make some columns sample uniformly in doamin of [min, max].

> generator.generate_uniform_random_dataset(dataset_description_file, N=10) # will generate a totoally random dataset.

Here is an example,
- generate 10 rows in synthetic datset
- "age" and "education" are sampled uniformly
- differential privacy parameter $\epsilon=0.01$

In [9]:
generator.generate_synthetic_dataset(dataset_description_file, N=10, epsilon=0.01, uniform_columns={'age', 'education'})

##### Step 7: Random missing

Remove values of given columns randomly, e.g., removing 60% of age values and 30% of race values.

In [10]:
generator.random_missing_on_columns(columns=['age', 'race'], missing_rates=[0.6, 0.3])

In [11]:
generator.synthetic_dataset

,ID,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,22638,NaN,Private,74382,HS-grad,13.0,Married-civ-spouse,Prof-specialty,Not-in-family,White,Female,326,14,16,POerdjUJcvWpUvS,<=50K
1,1852,48.0,Private,524818,1st-4th,9.0,Married-civ-spouse,Machine-op-inspct,Husband,NaN,Male,625,3458,38,kHfgKWSGFEruBUrnA,<=50K
2,24186,80.0,?,145822,11th,9.0,Never-married,?,Husband,White,Male,5661,13,35,uSEMXXsHvsxeBa,<=50K
3,27825,NaN,Private,92117,5th-6th,10.0,Divorced,Exec-managerial,Husband,White,Male,2,1612,34,KsyioTbiISJUOBW,<=50K
4,7560,NaN,Private,144621,HS-grad,9.0,Never-married,Sales,Unmarried,White,Male,722,679,54,aPMYKjb,<=50K
5,5025,51.0,Private,84670,Bachelors,9.0,Married-civ-spouse,Transport-moving,Husband,Black,Male,67,3664,35,mtHUG,>50K
6,13629,NaN,Private,228615,Preschool,12.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,54,17,34,irDXoY,<=50K
7,18308,NaN,Private,1185199,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Husband,NaN,Male,499,20,36,yoPfcqAFlxdA,>50K
8,8636,76.0,Private,154565,1st-4th,13.0,Divorced,Adm-clerical,Husband,NaN,Male,511,10,44,spbc,<=50K
9,420,NaN,?,105809,Masters,9.0,Married-civ-spouse,?,Husband,White,Female,72,14,36,ohO,>50K


##### Step 8: Save the synthetic dataset

In [12]:
generator.synthetic_dataset.to_csv(synthetic_dataset_file, index=False)